In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
from numpy.linalg import norm
from scipy import stats
import time

This K-Nearest Neighbor classifier is implemented to classify images of digits. Given vectors and a corresponding label.The data files are in ASCII text format, and each line of the files contains a feature vector of size 784, followed by its label. The coordinates of the feature vector are separated by spaces.

In [4]:
#Function to clean the txt files and seperate the labels from the actual vectors
def clean_txt(path):
    f = open(path,'r')
    arr = (f.read()).split("\n")
    arr = arr[:-1]
    labels = []
    for vector in arr:
        labels.append(int(vector[-1]))
    vectors = []
    for vector in arr:
        vectors.append(vector.split(' '))
    for i in range(len(vectors)):
        vectors[i] = list(map(int,vectors[i][:-1]))
    return np.array(vectors),np.array(labels)

In [5]:
test_vectors,test_labels = clean_txt("pa1test.txt")
train_vectors,train_labels = clean_txt("pa1train.txt")
vali_vectors,vali_labels = clean_txt("pa1validate.txt")

k_arr = [1,5,9,15]
test_k = [3] #Tested on training data and got 0.043 as the error.

### KNN Algorithm

In [6]:
def knn(vectors1, train_labels, vectors2, k):
    output_labels = np.array([])
    for test_vect in vectors2:
        labels = np.array([0]*k)
        vect_arr = [1]*k
        distances = np.array([np.Inf]*k)
        vect_index = 0
        for train_vect in vectors1:
            dist = norm(test_vect-train_vect)
            if dist < distances.max():
                index_to_change = np.where(distances == distances.max())[0][0]
                distances[index_to_change] = dist
                vect_arr[index_to_change] = train_vect
                labels[index_to_change] = train_labels[vect_index]
            vect_index += 1
        output_labels = np.append(output_labels, stats.mode(labels)[0][0])
    return output_labels

#### Testing k = 3

In [7]:
knn_3 = knn(train_vectors,train_labels,train_vectors,3)

In [8]:
num_wrong_train = 0
for i in range(len(knn_3)):
    if knn_3[i] != train_labels[i]:
        num_wrong_train +=1

In [9]:
num_wrong_train / len(train_vectors)

0.0435

In [ ]:
start1 = time.time()
train_error = []
vali_error = []
for k in k_arr:
    knn_k_train = knn(train_vectors, train_labels, train_vectors, k)
    knn_k_vali = knn(train_vectors, train_labels, vali_vectors, k)
    num_wrong_train = 0
    for i in range(len(knn_k_train)):
        if knn_k_train[i] != train_labels[i]:
            num_wrong_train +=1
    num_wrong_vali = 0
    for i in range(len(knn_k_vali)):
        if knn_k_vali[i] != vali_labels[i]:
            num_wrong_vali +=1
    train_error.append(num_wrong_train / len(train_vectors))
    vali_error.append(num_wrong_vali / len(knn_k_vali))
end1 = time.time()

In [ ]:
pd.DataFrame({"Train_Error":train_error, "Validation_Error":vali_error}, index = k_arr)

Seems like K = 1 performs the best on the Validation data, as the error is only 0.082

In [ ]:
knn_test = knn(train_vectors,train_labels,test_vectors,1)

In [ ]:
num_wrong_test = 0
for i in range(len(knn_test)):
    if knn_test[i] != test_labels[i]:
        num_wrong_test +=1

In [ ]:
num_wrong_test / len(test_labels)

So for K=1, the test error is 0.094.

Assessing how pre-processing steps affect the accuracy and running-time of a nearest neighbor classification algorithm.

In [ ]:
file = open("projection.txt","r")
file_arr = file.read().split("\n")[:-1]
proj_vectors = []
for string in file_arr:
    proj_vectors.append(string.split(" "))
for i in range(len(proj_vectors)):
    proj_vectors[i] = list(map(float,proj_vectors[i]))

In [ ]:
proj_train_vectors = np.matmul(train_vectors,proj_vectors)
proj_test_vectors = np.matmul(test_vectors,proj_vectors)
proj_vali_vectors = np.matmul(vali_vectors,proj_vectors)

In [ ]:
knn_proj_3 = knn(proj_train_vectors,train_labels,proj_train_vectors,3)

In [ ]:
num_wrong_proj_train = 0
for i in range(len(knn_3)):
    if knn_proj_3[i] != train_labels[i]:
        num_wrong_proj_train +=1

In [ ]:
error_k3 = num_wrong_proj_train / len(proj_train_vectors)
print("Error for K=3 is {}".format(error_k3))

In [ ]:
start2 = time.time()
proj_train_error = []
proj_vali_error = []
for k in k_arr:
    knn_k_proj_train = knn(proj_train_vectors, train_labels, proj_train_vectors, k)
    knn_k_proj_vali = knn(proj_train_vectors, train_labels, proj_vali_vectors, k)
    num_wrong_proj_train = 0
    for i in range(len(knn_k_proj_train)):
        if knn_k_proj_train[i] != train_labels[i]:
            num_wrong_proj_train +=1
    num_wrong_proj_vali = 0
    for i in range(len(knn_k_proj_vali)):
        if knn_k_proj_vali[i] != vali_labels[i]:
            num_wrong_proj_vali +=1
    proj_train_error.append(num_wrong_proj_train / len(proj_train_vectors))
    proj_vali_error.append(num_wrong_proj_vali / len(knn_k_proj_vali))
end2 = time.time()

In [ ]:
pd.DataFrame({"Projected_Train_Error":proj_train_error, "Projected_Validation_Error":proj_vali_error}, index = k_arr)

In [ ]:
proj_knn_test = knn(proj_train_vectors,train_labels,proj_test_vectors,15)
num_wrong_proj_test = 0
for i in range(len(proj_knn_test)):
    if proj_knn_test[i] != test_labels[i]:
        num_wrong_proj_test +=1
print("Error for K=15 is {}".format(num_wrong_proj_test / len(test_labels)))

In [ ]:
print("Time for part 1: {}".format(end1-start1))
print("Time for part 2: {}".format(end2-start2))

After projection, the classifier accuarcy is worse, but it is somewhat faster since we are projecting into a lower dimension as opposed to the higher dimensions earlier. 